In [25]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'd0846215-0c0f-4fcb-82a7-7755f505c9d3',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url)
  dataCMC = json.loads(response.text)

  print(dataCMC['data'][1002])
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'id': 3639, 'name': 'PlayGame', 'symbol': 'PXG', 'slug': 'playgame', 'rank': 1884, 'is_active': 1, 'first_historical_data': '2018-12-18T20:44:37.000Z', 'last_historical_data': '2022-10-06T09:19:00.000Z', 'platform': {'id': 1027, 'name': 'Ethereum', 'symbol': 'ETH', 'slug': 'ethereum', 'token_address': '0x47e67ba66b0699500f18a53f94e2b9db3d47437e'}}


In [56]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/info?slug=tether'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'd0846215-0c0f-4fcb-82a7-7755f505c9d3',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url)
  data_draft = json.loads(response.text)

  print(data_draft)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'status': {'timestamp': '2022-10-06T10:17:10.277Z', 'error_code': 0, 'error_message': None, 'elapsed': 27, 'credit_count': 1, 'notice': None}, 'data': {'825': {'id': 825, 'name': 'Tether', 'symbol': 'USDT', 'category': 'token', 'description': 'Tether (USDT) is a cryptocurrency and operates on the Ethereum platform. Tether has a current supply of 70,155,449,905.79822 with 68,212,574,445.55758 in circulation. The last known price of Tether is 1.00011639 USD and is down -0.01 over the last 24 hours. It is currently trading on 40520 active market(s) with $45,106,167,099.51 traded over the last 24 hours. More information can be found at https://tether.to.', 'slug': 'tether', 'logo': 'https://s2.coinmarketcap.com/static/img/coins/64x64/825.png', 'subreddit': '', 'notice': '', 'tags': ['payments', 'stablecoin', 'asset-backed-stablecoin', 'avalanche-ecosystem', 'solana-ecosystem', 'arbitrum-ecosytem', 'moonriver-ecosystem', 'injective-ecosystem', 'bnb-chain', 'usd-stablecoin'], 'tag-names': [

In [34]:
data_draft['data']['825']['contract_address']
for data in data_draft['data']['825']['contract_address']:
  print(data)

{'contract_address': '0xdac17f958d2ee523a2206206994597c13d831ec7', 'platform': {'name': 'Ethereum', 'coin': {'id': '1027', 'name': 'Ethereum', 'symbol': 'ETH', 'slug': 'ethereum'}}}
{'contract_address': 'TR7NHqjeKQxGTCi8q8ZY4pL8otSzgjLj6t', 'platform': {'name': 'Tron20', 'coin': {'id': '1958', 'name': 'TRON', 'symbol': 'TRX', 'slug': 'tron'}}}
{'contract_address': 'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB', 'platform': {'name': 'Solana', 'coin': {'id': '5426', 'name': 'Solana', 'symbol': 'SOL', 'slug': 'solana'}}}
{'contract_address': '312769', 'platform': {'name': 'Algorand', 'coin': {'id': '4030', 'name': 'Algorand', 'symbol': 'ALGO', 'slug': 'algorand'}}}
{'contract_address': '0xa71edc38d189767582c38a3145b5873052c3e47a', 'platform': {'name': 'HECO', 'coin': {'id': '2502', 'name': 'Huobi Token', 'symbol': 'HT', 'slug': 'huobi-token'}}}
{'contract_address': '0x55d398326f99059ff775485246999027b3197955', 'platform': {'name': 'BNB Smart Chain (BEP20)', 'coin': {'id': '1839', 'name': 

In [49]:
list(data_draft['data'][list(data_draft['data'])[0]])

['id',
 'name',
 'symbol',
 'category',
 'description',
 'slug',
 'logo',
 'subreddit',
 'notice',
 'tags',
 'tag-names',
 'tag-groups',
 'urls',
 'platform',
 'date_added',
 'twitter_username',
 'is_hidden',
 'date_launched',
 'contract_address',
 'self_reported_circulating_supply',
 'self_reported_tags',
 'self_reported_market_cap']

In [35]:
compact_dataCMC = []
ids = []
for coin_data in dataCMC['data']:
  ids.append(coin_data['id'])
  if coin_data['platform'] != None:
    pfs = coin_data['platform']
    # print(pfs)
    compact_dataCMC.append({
      'slug': coin_data['slug'],
      'address': pfs['token_address'],
      'chain': pfs['symbol'],
      'id': coin_data['id']
    })
compact_dataCMC[:10]

[{'slug': 'belacoin',
  'address': '0x2e98a6804e4b6c832ed0ca876a943abd3400b224',
  'chain': 'ETH',
  'id': 217},
 {'slug': 'gamecredits',
  'address': '0x63f88a2298a5c4aee3c216aa6d926b184a4b2437',
  'chain': 'ETH',
  'id': 576},
 {'slug': 'circuits-of-value',
  'address': '0x3d658390460295fb963f54dc0899cfb1c30776df',
  'chain': 'ETH',
  'id': 788},
 {'slug': 'tether',
  'address': '0xdac17f958d2ee523a2206206994597c13d831ec7',
  'chain': 'ETH',
  'id': 825},
 {'slug': 'xaurum',
  'address': '0x4DF812F6064def1e5e029f1ca858777CC98D2D81',
  'chain': 'ETH',
  'id': 895},
 {'slug': 'agoras-tokens',
  'address': '0x738865301a9b7dd80dc3666dd48cf034ec42bdda',
  'chain': 'ETH',
  'id': 1037},
 {'slug': 'augur',
  'address': '0x1985365e9f78359a9B6AD760e32412f4a445E862',
  'chain': 'ETH',
  'id': 1104},
 {'slug': 'digixdao',
  'address': '0xe0b7927c4af23765cb51314a0e0521a9645f0e2a',
  'chain': 'ETH',
  'id': 1229},
 {'slug': 'pluton',
  'address': '0xd8912c10681d8b21fd3742244f44658dba12264e',
  'c

In [36]:
len(compact_dataCMC), len(ids)

(8111, 9442)

In [37]:
ids

[1,
 2,
 3,
 4,
 5,
 6,
 8,
 10,
 13,
 16,
 18,
 25,
 35,
 37,
 41,
 42,
 43,
 45,
 52,
 53,
 56,
 61,
 64,
 66,
 67,
 69,
 72,
 74,
 77,
 78,
 80,
 83,
 87,
 90,
 93,
 99,
 109,
 118,
 120,
 122,
 128,
 131,
 132,
 141,
 145,
 148,
 151,
 154,
 162,
 168,
 170,
 182,
 184,
 201,
 213,
 215,
 217,
 254,
 258,
 260,
 263,
 268,
 276,
 278,
 290,
 291,
 293,
 298,
 313,
 316,
 328,
 331,
 333,
 360,
 362,
 366,
 367,
 372,
 377,
 389,
 405,
 416,
 460,
 463,
 470,
 501,
 502,
 506,
 512,
 541,
 551,
 558,
 572,
 576,
 584,
 588,
 597,
 601,
 624,
 633,
 638,
 644,
 656,
 659,
 693,
 702,
 703,
 707,
 720,
 730,
 733,
 760,
 764,
 788,
 789,
 799,
 815,
 819,
 825,
 831,
 833,
 837,
 853,
 857,
 859,
 873,
 894,
 895,
 898,
 911,
 916,
 918,
 921,
 934,
 938,
 945,
 948,
 951,
 977,
 978,
 986,
 993,
 1004,
 1019,
 1020,
 1026,
 1027,
 1032,
 1033,
 1035,
 1037,
 1038,
 1042,
 1044,
 1052,
 1053,
 1066,
 1070,
 1082,
 1085,
 1087,
 1090,
 1104,
 1106,
 1107,
 1120,
 1135,
 1136,
 1146,
 1

In [58]:
import time

In [60]:
count = 0
_id = ''
cmc_data = []
for id in ids:
  count += 1
  # if count % 100 == 0:
  if count % 100 == 0 or count == 9442:
    time.sleep(3)
    _id = _id + str(id)
    url = f'https://pro-api.coinmarketcap.com/v2/cryptocurrency/info?id={_id}'.format(_id)
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': 'd0846215-0c0f-4fcb-82a7-7755f505c9d3',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url)
      # data_draft['data'][list(data_draft['data'])[0]]
      fetched_data = json.loads(response.text)
      for coin in fetched_data['data']:
        coin_data = fetched_data['data'][coin]
          # if coin_data['platform'] != None:
          #   pfs = coin_data['platform']
          #   # print(pfs)
          #   compact_dataCMC.append({
          #     'slug': coin_data['slug'],
          #     'address': pfs['token_address'],
          #     'chain': pfs['symbol'],
          #     'id': coin_data['id']
          #   })
        cmc_data.append({
          'slug': coin_data['slug'],
          'contract_address': coin_data['contract_address']
        })
      # print(cmc_data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    _id =''
  else:
    _id = _id + str(id) +  ','
    


In [61]:
len(cmc_data)

9442

In [62]:
with open('data.json', 'w') as outfile:
      json.dump(cmc_data, outfile, indent=4)

In [69]:
data_BNB = []
data_ETH = []
data_FTM = []
data_MATIC = []
for coin in cmc_data:
  for pfs in coin['contract_address']:
    if pfs['platform']['coin']['symbol'] == 'BNB':
      data_BNB.append({
        'slug': coin['slug'],
        'address': pfs['contract_address']
      })
    if pfs['platform']['coin']['symbol'] == 'ETH' :
      data_ETH.append({
        'slug': coin['slug'],
        'address': pfs['contract_address']
      })
    if pfs['platform']['coin']['symbol'] == 'FTM':
      data_FTM.append({
        'slug': coin['slug'],
        'address': pfs['contract_address']
      })
    if pfs['platform']['coin']['symbol'] == 'MATIC':
      data_MATIC.append({
        'slug': coin['slug'],
        'address': pfs['contract_address']
      })

In [70]:
len(data_BNB), len(data_ETH), len(data_FTM), len(data_MATIC)

(4233, 3557, 222, 722)

In [71]:
data_BNB[0]

{'slug': 'litecoin', 'address': '0x4338665cbb7b2485a8855a139b75d5e34ab0db94'}

In [40]:
import json 

f = open('origin/bsc_tokens.json')
data = json.load(f)
odata_BNB = data.copy()

print(odata_BNB[0])

{'coin_id': 'binancecoin', 'name': 'Binance Coin', 'symbol': 'BNB', 'address': '0x', 'decimals': 18, 'market_cap_rank': 5, 'total_volume': 53927152493, 'market_cap': 65974526709, 'price': 478.12}


In [76]:
def augment_data(file_name, data):
  f = open(f'origin/{file_name}.json'.format(file_name))
  odata = json.load(f)
  print(len(odata))

  augmented_data = []
  unaugmented_data = []
  for odoc in odata:
    augmented = False
    for doc in data:
      if odoc['address'] == doc['address']:
        odoc['cmc_id'] = doc['slug']
        augmented = True 
        augmented_data.append(odoc)
    if not augmented:
      unaugmented_data.append(odoc) 
  print(len(augmented_data), len(unaugmented_data))
    
  with open(f'augmented/{file_name}.json'.format(file_name), 'w') as outfile:
    json.dump(augmented_data, outfile, indent=4)
  with open(f'unaugmented/{file_name}.json'.format(file_name), 'w') as outfile:
    json.dump(unaugmented_data, outfile, indent=4)

In [77]:
augment_data('bsc_tokens', data_BNB)
augment_data('eth_tokens', data_ETH)
augment_data('ftm_tokens', data_FTM)
augment_data('polygon_tokens', data_MATIC)

1360
539 821
3232
1377 1855
83
24 59
23
9 14
